<a href="https://colab.research.google.com/github/kaushaln1/ML_DM_labs/blob/main/DM/DM_HW_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error,mean_absolute_percentage_error,mean_squared_log_error

from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer ,make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import Ridge
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DM3/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DM3/test.csv')

# Preprocessing

In [ ]:
column_map = {}


for column in df_train.columns:
    unique_vals = df_train[column].unique()
    data_type = df_train[column].dtype

    if len(unique_vals) == 2:
        column_map[column] = 'binary'
        continue

    if data_type == 'object' and len(unique_vals) > 2:
        column_map[column] = 'ordinal'
        continue

    if data_type in ['int64', 'float64']:
        column_map[column] = 'numerical'
        continue


# Normalized data

In [ ]:
# normalize numerical attributes
numerical_atrributes = []
for key, value in column_map.items():
        if value == 'numerical':
          numerical_atrributes.append(key)
numerical_atrributes
df_train_norm_filtered = df_train.filter(numerical_atrributes, axis=1)
df_train_norm_filtered
df_tb_nor = df_train_norm_filtered.dropna()

d= preprocessing.normalize(df_tb_nor[numerical_atrributes], axis=0) # normalizing all attributes
normalised_train_data = pd.DataFrame(d, columns=df_tb_nor.columns)


# normalize test data
df_test_norm_filtered = df_test.filter(numerical_atrributes, axis=1)

df_test_nor = df_test_norm_filtered.dropna()

d= preprocessing.normalize(df_test_nor[numerical_atrributes], axis=0) # normalizing all attributes
normalised_test_data = pd.DataFrame(d, columns=df_test_nor.columns)

# Selection of features

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
sfs = SequentialFeatureSelector(knn, n_features_to_select=3)
X_train = normalised_train_data.drop(columns=['SalePrice'])
y_train = normalised_train_data['SalePrice']
sfs.fit(X_train, y_train)
sfs.transform(X_train).shape


ValueError: ignored

In [ ]:
feature_names = normalised_train_data.drop(columns=['SalePrice']).columns.to_list()
X = normalised_train_data.drop(columns=['SalePrice']).values
y = normalised_train_data['SalePrice'].values


selected_features = []
remaining_features = list(range(X.shape[1]))
best_features = []
best_performance = float('inf')

num_features_to_select = 10

kfold = KFold(n_splits=10)

while len(selected_features) < num_features_to_select and remaining_features:
    best_feature = None
    for feature in remaining_features:
        features_to_test = selected_features + [feature]
        X_subset = X[:, features_to_test]

        fold_errors = []
        for train_idx, val_idx in kfold.split(X_subset):
            X_train, X_val = X_subset[train_idx], X_subset[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            # Train the model
            model = LinearRegression()
            model.fit(X_train, y_train)

            # Evaluate on the validation set
            y_pred = model.predict(X_val)
            fold_error = mean_squared_error(y_val, y_pred)
            fold_errors.append(fold_error)

        mean_error = np.mean(fold_errors)
        if mean_error < best_performance:
            best_performance = mean_error
            best_feature = feature

    if best_feature is not None:
        selected_features.append(best_feature)
        remaining_features.remove(best_feature)
        best_features.append(best_feature)

selected_feature_names = [feature_names[idx] for idx in selected_features]
print("Selected Features:", selected_feature_names)

Selected Features: ['OverallQual', 'GrLivArea', 'BsmtFinSF1', 'GarageArea', 'YearRemodAdd', 'MSSubClass', 'MasVnrArea', 'Fireplaces', 'TotalBsmtSF', 'YearBuilt']


In [ ]:
selected_feature_names.append('SalePrice')
df_train_best_10_features = normalised_train_data.filter(selected_feature_names,axis=1)
df_test_best_10_features  = normalised_test_data.filter(selected_feature_names, axis =1 )

In [ ]:
# using numerical attributes
X_train = df_train_best_10_features.drop(columns=['SalePrice'])
y_train = df_train_best_10_features['SalePrice']

X_test = df_test_best_10_features.drop(columns=['SalePrice'])
y_test = df_test_best_10_features['SalePrice']

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

y_pred_train = model.predict(X_train)
# create csv with
mse = mean_squared_error(y_test, y_pred)
mse2 = mean_squared_error(y_train, y_pred_train)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error (MSE) Train Error:  {mse2}")
print(f"Mean Squared Error (MSE) Test Error: {mse}")


Mean Squared Error (MSE) Train Error:  3.826873504952882e-05
Mean Squared Error (MSE) Test Error: 0.009589233903404918


# Polynomial Transform

**Question 2 **

In [ ]:
poly = PolynomialFeatures(degree=5)
X_train_poly = poly.fit_transform(X_train)

X_test_poly = poly.transform(X_test)

print("X_train shape before -> after  transform" , X_train.shape, "-->", X_train_poly.shape)
print("X_test  shape before -> after transform", X_test.shape, "-->", X_test_poly.shape)

X_train shape before -> after  transform (766, 10) --> (766, 3003)
X_test  shape before -> after transform (355, 10) --> (355, 3003)


**Question 3 **

In [ ]:
model = LinearRegression()
model.fit(X_train_poly, y_train)


y_train_pred = model.predict(X_train_poly)
y_test_pred = model.predict(X_test_poly)

train_error = mean_squared_error(y_train, y_train_pred)
test_error = mean_squared_error(y_test, y_test_pred)
test_e = np.mean(abs(y_test-y_test_pred))
print("Train Error(MSE) on Polynomially transformed data(deg=5) :", train_error)
print("Test Error (MSE) on Polynomially Transformed data(deg=5) :", test_e)

Train Error(MSE) on Polynomially transformed data(deg=5) : 1.0209895058632275e-08
Test Error (MSE) on Polynomially Transformed data(deg=5) : 184.51528357862554


# Question 4

In [ ]:
reg_values = [0.001, 0.01, 0.1, 1, 5, 10, 20, 50, 100, 1000]

cv_errors = []
for reg_p in reg_values:
    ridge = Ridge(alpha=reg_p)
    scores = cross_val_score(ridge, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
    cv_errors.append(np.mean(-scores))
print ("///-----------------------------------Reg_parameter and ECv --------------------------------------------///")
for reg_p, error in zip(reg_values, cv_errors):
    print(f"\t Regularization Parameter : {reg_p},  Cross-Validation Error: {error:.9f}")

chosen_reg_p = 10


ridge = Ridge(alpha=chosen_reg_p)
ridge.fit(X_train, y_train)
yhat_train = ridge.predict(X_train)
yhat_test = ridge.predict(X_test)


train_error = np.mean(( yhat_train - y_train) ** 2)
test_error = np.mean(( yhat_test - y_test) ** 2)
print ("///-----------------------------------------------------------------------------------------------------------///")
print(f"Selected Value of Regularization Paramneter : {chosen_reg_p}")
print(f"Training Error with Reg_p {chosen_reg_p} is : {train_error}")
print(f"Test Error with Reg_p {chosen_reg_p} is : {test_error}")

///-----------------------------------Reg_parameter and ECv --------------------------------------------///
	 Regularization Parameter : 0.001,  Cross-Validation Error: 0.000043063
	 Regularization Parameter : 0.01,  Cross-Validation Error: 0.000044918
	 Regularization Parameter : 0.1,  Cross-Validation Error: 0.000061588
	 Regularization Parameter : 1,  Cross-Validation Error: 0.000123446
	 Regularization Parameter : 5,  Cross-Validation Error: 0.000186151
	 Regularization Parameter : 10,  Cross-Validation Error: 0.000202848
	 Regularization Parameter : 20,  Cross-Validation Error: 0.000212799
	 Regularization Parameter : 50,  Cross-Validation Error: 0.000219395
	 Regularization Parameter : 100,  Cross-Validation Error: 0.000221710
	 Regularization Parameter : 1000,  Cross-Validation Error: 0.000223845
///-----------------------------------------------------------------------------------------------------------///
Selected Value of Regularization Paramneter : 10
Training Error with Re

# Question 5

In [ ]:

mlp = MLPRegressor(max_iter=1000)

parameters = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],
    'alpha': [0.0001, 0.001, 0.01],
}

kfold = KFold(n_splits=10, shuffle=True, random_state=42)

grid_search = GridSearchCV(mlp, parameters, cv=kfold, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)


best_model = grid_search.best_estimator_

results = grid_search.cv_results_
print("Mean cross-validation Error for all models:")
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"ECv {mean_score:.9f}, Parameters: {params}")


print("\nOptimal Model Parameters  : " , grid_search.best_params_)
print("Best cross-validation Error (negative MSE):", grid_search.best_score_)


best_model.fit(X_train, y_train)

train_pred = best_model.predict(X_train)
test_pred = best_model.predict(X_test)

train_error = mean_squared_error(y_train, train_pred)
test_error = mean_squared_error(y_test, test_pred)


print(f"Train MSE with regp {grid_search.best_params_.get('alpha')} is :", train_error)
print(f"Test MSE with regp {grid_search.best_params_.get('alpha')} is:", test_error)




Mean cross-validation Error for all models:
ECv -0.000371012, Parameters: {'alpha': 0.0001, 'hidden_layer_sizes': (50,)}
ECv -0.000195479, Parameters: {'alpha': 0.0001, 'hidden_layer_sizes': (100,)}
ECv -0.000202535, Parameters: {'alpha': 0.0001, 'hidden_layer_sizes': (50, 50)}
ECv -0.000141606, Parameters: {'alpha': 0.0001, 'hidden_layer_sizes': (100, 100)}
ECv -0.000271681, Parameters: {'alpha': 0.001, 'hidden_layer_sizes': (50,)}
ECv -0.000196039, Parameters: {'alpha': 0.001, 'hidden_layer_sizes': (100,)}
ECv -0.000158889, Parameters: {'alpha': 0.001, 'hidden_layer_sizes': (50, 50)}
ECv -0.000113189, Parameters: {'alpha': 0.001, 'hidden_layer_sizes': (100, 100)}
ECv -0.000311873, Parameters: {'alpha': 0.01, 'hidden_layer_sizes': (50,)}
ECv -0.000199820, Parameters: {'alpha': 0.01, 'hidden_layer_sizes': (100,)}
ECv -0.000215858, Parameters: {'alpha': 0.01, 'hidden_layer_sizes': (50, 50)}
ECv -0.000105994, Parameters: {'alpha': 0.01, 'hidden_layer_sizes': (100, 100)}

Optimal Model Pa

# Question 6

In [ ]:
#classification data
df_train_class = df_train.copy()
df_train_class['Class'] = df_train_class['SalePrice'].apply(lambda x: 1 if x <= 150000 else -1)
df_train_class.drop('SalePrice', axis=1)

df_test_class = df_test.copy()
df_test_class['Class'] = df_test_class['SalePrice'].apply(lambda x: 1 if x <= 150000 else -1)
df_test_class.drop('SalePrice', axis=1)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Class
0,1,20,RL,80.0,11900,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,-1
1,2,90,RM,78.0,7060,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,11,2008,WD,Alloca,-1
2,3,60,RL,NaN,7851,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,-1
3,4,60,RL,NaN,11000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2007,WD,Normal,-1
4,5,180,RM,35.0,3675,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,456,190,RL,70.0,7000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,1
456,457,50,RL,75.0,45600,Pave,NaN,IR2,Bnk,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,-1
457,458,20,RL,NaN,11717,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2009,WD,Normal,-1
458,459,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,-1


In [ ]:
df_train_class

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Class
0,1,120,RL,30.0,9549,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,4,2006,WD,Normal,270000,-1
1,2,45,RH,60.0,9000,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,10,2009,WD,Normal,76000,1
2,3,60,RL,NaN,12394,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,10,2007,WD,Family,225000,-1
3,4,20,RL,75.0,11957,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,7,2008,WD,Normal,232000,-1
4,5,20,RL,NaN,11616,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2009,WD,Normal,139000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,50,RM,50.0,6000,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,7,2009,WD,Normal,128000,1
996,997,20,RL,60.0,6960,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,Shed,500,11,2009,WD,Normal,120500,1
997,998,20,RL,78.0,10206,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,10,2008,WD,Normal,245000,-1
998,999,60,RL,NaN,11214,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,7,2006,WD,Normal,199900,-1


In [ ]:
df_train_norm_filtered

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,120,30.0,9549,8,5,1995,1996,0.0,437,...,0,30,0,0,216,0,0,4,2006,270000
1,2,45,60.0,9000,6,3,1928,1950,0.0,0,...,0,0,91,0,0,0,0,10,2009,76000
2,3,60,NaN,12394,7,5,2003,2003,0.0,0,...,100,48,0,0,0,0,0,10,2007,225000
3,4,20,75.0,11957,8,5,2006,2006,53.0,24,...,144,104,0,0,0,0,0,7,2008,232000
4,5,20,NaN,11616,5,5,1962,1962,116.0,170,...,0,20,144,0,0,0,0,9,2009,139000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,50,50.0,6000,4,6,1936,1950,0.0,672,...,0,0,0,0,0,0,0,7,2009,128000
996,997,20,60.0,6960,4,6,1970,1970,0.0,375,...,96,0,0,0,0,0,500,11,2009,120500
997,998,20,78.0,10206,8,5,2007,2007,468.0,33,...,144,99,0,0,0,0,0,10,2008,245000
998,999,60,NaN,11214,7,5,1998,1999,0.0,0,...,89,0,0,0,0,0,0,7,2006,199900


In [ ]:
df_test_norm_filtered

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,20,80.0,11900,6,5,1957,1957,387.0,1040,...,0,0,0,0,192,0,0,6,2008,166000
1,2,90,78.0,7060,7,5,1997,1998,200.0,1309,...,0,0,0,0,0,0,0,11,2008,206300
2,3,60,NaN,7851,6,5,2002,2002,NaN,625,...,288,48,0,0,0,0,0,5,2010,216500
3,4,60,NaN,11000,8,5,2000,2000,72.0,0,...,0,56,0,0,0,0,0,6,2007,248000
4,5,180,35.0,3675,6,5,2005,2005,80.0,459,...,0,28,0,0,0,0,0,6,2008,148000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,456,190,70.0,7000,5,7,1910,1991,0.0,969,...,85,0,148,0,0,0,0,1,2008,107500
456,457,50,75.0,45600,6,8,1908,1997,0.0,0,...,486,40,0,0,175,0,0,9,2008,240000
457,458,20,NaN,11717,6,6,1970,1970,571.0,0,...,371,0,0,0,0,0,0,2,2009,185000
458,459,70,66.0,9042,7,9,1941,2006,0.0,275,...,0,60,0,0,0,0,2500,5,2010,266500


In [ ]:
X_train

,OverallQual,GrLivArea,BsmtFinSF1,GarageArea,YearRemodAdd,MSSubClass,MasVnrArea,Fireplaces,TotalBsmtSF,YearBuilt
0,0.045134,0.033232,0.025377,0.032285,0.036310,0.061385,0.000000,0.04222,0.046697,0.036535
1,0.033850,0.017439,0.000000,0.024163,0.035473,0.023019,0.000000,0.00000,0.024505,0.035308
2,0.045134,0.035011,0.001394,0.055307,0.036492,0.010231,0.008637,0.04222,0.049197,0.036737
3,0.039492,0.033921,0.081298,0.036312,0.036383,0.010231,0.039763,0.00000,0.047666,0.036627
4,0.039492,0.036257,0.000000,0.030271,0.036401,0.010231,0.027866,0.04222,0.050948,0.036627
...,...,...,...,...,...,...,...,...,...,...
761,0.016925,0.023133,0.000000,0.026848,0.035473,0.046039,0.000000,0.00000,0.000000,0.035711
762,0.022567,0.029450,0.039023,0.016109,0.035473,0.025577,0.000000,0.00000,0.021004,0.035455
763,0.022567,0.019218,0.021776,0.044300,0.035837,0.010231,0.000000,0.00000,0.027005,0.036078
764,0.045134,0.034766,0.001916,0.050877,0.036510,0.010231,0.076266,0.04222,0.048853,0.036755
